In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasetsforecast.m4 import M4 

from neuralforecast.auto import AutoAutoformer
from neuralforecast.tsdataset import TimeSeriesDataset

In [10]:
# load m4
df = M4.load(
            directory="C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m4",
            group="Monthly")[0]

def train_test_split(df: pd.DataFrame, horizon: int):
    """Split the dataframe into training and test sets by horizon."""
    groups = df.groupby('unique_id')
    train_list, test_list = [], []
    for _, group_df in groups:
        group_df = group_df.sort_values('ds')
        train_list.append(group_df.head(-horizon))
        test_list.append(group_df.tail(horizon))
    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True)
    return train_df, test_df

df['ds'] = pd.to_datetime(df['ds']).astype(int)

y_train_df, y_test_df = train_test_split(df, horizon=18)

dataset, *_ = TimeSeriesDataset.from_df(y_train_df)

In [ ]:
model = AutoAutoformer(h=18, num_samples=20, backend="optuna")

model.fit(dataset)

[I 2025-03-04 21:00:24,247] A new study created in memory with name: no-name-91486902-abd1-4558-9eff-d76ae9ac400c
c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\neuralforecast\common\_base_auto.py:291: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  v = trial.suggest_loguniform(k, v.lower, v.upper)
INFO:lightning_fabric.utilities.seed:Seed set to 20
[W 2025-03-04 21:00:24,282] Trial 0 failed with parameters: {'hidden_size': 128, 'n_head': 8, 'learning_rate': 0.0019179140448390226, 'scaler_type': None, 'max_steps': 2000, 'batch_size': 256, 'windows_batch_size': 256, 'random_seed': 20, 'input_size': 72, 'step_size': 1} because of the following error: AttributeError("'DataFrame' object has no attribute 'temporal_cols'").
Traceback (most recent call last):
  File "c:\Users\ricar\mixture_of_experts_time_series\ve

AttributeError: 'DataFrame' object has no attribute 'temporal_cols'

In [ ]:
# get best model config
test_dataset, *_ = TimeSeriesDataset.from_df(y_test_df)

y_hat = model.predict(dataset=test_dataset)

y_hat